
* To begin, it's best to understand what we're trying to implement and model here.
The two models which I'll be working with are both recomendation focused, but they have different approaches to doing it. The first is content-based recommendation and the second is collaborative-filtering recommendation.
- Content-based is what it sounds like: recommendating different, often new things to a user based on the similairity of content between the user's past matreial consumption.
- Collaborative-filtering is also what it sounds like: reocmmendating different, new things to a user based off the comsuption material of other users which might share similer preferences or behaviors.
** So with some understanding of the model types to be used, what sort of data might we use with them?
To start, it's correct to assume that such a set of data would be made up of users and their data in order for recomendation to take place: but such a nice neat dataset isn't avaible, so it has to be made much like it would in product.

*** To create our dataset, I'll be using the three given within canves.
- links_small.csv: a set for the imbdb and tmdbld ids for moives.
- ratings_small.csv: user's ids and their rating of moives with repsect to their IDs and time the review was done.
- movies_metadata.cvs: data about moives, connecting to their repsectives IDs.

  This really all means I'll be following the notebook given, but there will be communtary from me along the way so as to boost my understanding.

* First I'll use content based recommendation for the act of doing so.
This means importting the needed packages and functions form such packages for user now and later

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None

from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD #(if this code doesn't run, please remove the "#" before pip install surprise in the previous cell and run the cell)
from surprise.model_selection import cross_validate


import warnings; warnings.simplefilter('ignore')



and now I need to load in the data, but doing so for the links_small and moives_metadata will require some modifcation to it's ID column.

In [3]:
small_mdf = pd.read_csv('links_small.csv')
small_mdf = small_mdf[small_mdf['tmbdId'].notnull()]['tmbdId'].astype('int')

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

m_df = pd.read_csv('movies_metadata.csv')
m_df.head()

m_df['id'] = m_df['id'].apply(convert_int)
m_df[m_df['id'].isnull()]

m_df = m_df.drop([19730, 29503, 35587])
m_df['id'] = m_df['id'].astype('int')

sm_df = m_df[m_df['id'].isin(small_mdf)]

sm_df.shape

sm_df['tagline'] = sm_df['tagline'].fillna('')
sm_df['description'] = sm_df['overview'] + sm_df['tagline']
sm_df['description'] = sm_df['description'].fillna('')

FileNotFoundError: [Errno 2] No such file or directory: 'links_small.csv'


so I have read the csv file using pandas which has created the contenets of the csv file into a dataframe; after this, a boolean series is used to filter out missing values within the 'tmdbId' colummn and then convert the remaining ones into interger data types, overwriting the orginal dataframe: so it can be seen that I'm only interested in that label with it's featuresi. Most of the action happens with the conjection-like method syntax that calls the other functions to the series/new dataframe.

What follows after is a defining of a function that will be used later to convert the 'id' label's features into interger data types after the label was identifyed with & converted to a dataframe with pd.read_csv. After this, a similer boolean series is used to create a sub-dataframe of only the 'id' label and applies the convert_int function to it, checking after is any feature is null: such features are, and their row is dropped by the '.drop' method  passed those IDs. After this is done, a new datafame of the consiting only of the 'id' lable is created, but to make sure it's accuracy another boolean series is created which checks the 'id' data values against the orginal dataframe, giving a list of true or false for each one; the ones found to be mis-matched are passed along to the sub-set dataframe 'm_df' and then assigned to the new, smaller main dataset/dataframe.

With this new smaller subset dataframe of the larger, orginal I can fill any taglines rows with an empty string via a 'fillna('') for the prepreation of it's use later; as well, I do this with the conjoined lable 'decription' made up of the lable's & their fatures's 'overview' and 'tagline'

** With the needed, smaller indexed values of the moive IDs created, I can move onto to futher defining thet type of 'content' I'll be using to and build the recomendation model.
The two main building blocks of the recommendation model will be the computions and results of the TF-IDF matrix and Cosine Similarity.

In [4]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(sm_df['description'])
tfidf_matrix.shape

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

sm_df = sm_df.reset_index()
titles = sm_df['title']
indices = pd.Series(sm_df.index, index=sm_df['title'])

def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:51]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

NameError: name 'sm_df' is not defined



* Moving onto collaborative based filtering.
The process for doing this is going to be sightly different and slicker then content based filtering. Instead of picking out one label from the dataset, I'm going to pick out three labels and pass them to a cross_vaildation function that takes in the singular value decomposition function, the data labels with features, and a defining of deisred parameters. From this, the output of the RMS will be passed to be review. After, the svd function will be used to predict the rating a user would give a ceritan moive via it's ID.

In [5]:
reader = Reader()

ratings = pd.read_csv('ratings_small.csv')
ratings.head()



data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

ratings[ratings['userId'] == 1]

svd.predict(1, 302, 3)

FileNotFoundError: [Errno 2] No such file or directory: 'ratings_small.csv'


* And now a hybrid model.
The beginning process for this is correcting the features of multiple columns which will be merged with index & indices being set. After this the hybrid model will be bulit setting objects, getting the sim_scores of titles, and running the svd.predict.

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(sm_df[['title', 'id']], on='id').set_index('title')

indices_map = id_map.set_index('id')

def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = sm_df.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

hybrid(1, 'Avatar')


* Conclusion.
For my review, I'm not going to focus mainly on what has happen in this notebook, but instead focus on those things I've learned to be an important part of the ovarall multi-model creation.

** Pandas was used heavily throughout each creation of a model type.
Much of it's use was towards isolating a single or multiple labels along with their features, sometimes converting them to interger types or creating indexes out of them. These isolatd lablels would then be made into it's own dataframe for model use.

** Why cosine similiarty, singular value decomposition, and TF-IDF is used.
Thought TF-IDF isn't used at the same time of singluar value decomposition within this notebook, I'm trying to think of them in the funciton in isolation to deduce their application in the three model types.

*** Cosine Similiaty
A metric used to find the cosine of the angle between two vectors in an inner product space; this means you can base similarity on if vectors are pointing in the same dirction, or any other angle repsective to each other indicated by a value between -1 (opposite directions) and 1(same direction)
*** Singular Value Decomposition
This transformer is used to linearly preform dimensionality reduction on a dataframe by the means of truncated singluar value decomposition.
*** TF-IDF
The complete symbol for this algorithm is Term Fequency and Inverse Doucment Frequency; this is to say it's the product of the term frequency of some term within the set of documents by the inverse of that term's frequency in total number of documents.

Looking it over, it would seem that cosine similiarty and TF-IDF are similer in nature but are used in conjunction with each other. This would seem to imply the product of TF-IDF can be used in conjunction with cosine similaty metrics.